In [1]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kolli\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#read excel
excel_file = "Input.xlsx"
df = pd.read_excel(excel_file)

In [ ]:
pip install openpyxl

In [ ]:
pip install nltk


In [1]:
# Function to scrape webpage and save content into a text file
def content_files(url_id, url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract title
    title_tag = soup.find('title')
    if title_tag:
        title = title_tag.get_text()
    else:
        title = "Title not found"
    
    # Extract content
    content = ""
    for paragraph in soup.find_all('p'):
        content += paragraph.get_text() + "\n"
    
    with open(f'extracted_files/{url_id}.txt', 'w', encoding='utf-8') as f:
        f.write(title + '\n\n')
        f.write(content)

In [4]:
if not os.path.exists('extracted_files'):
    os.makedirs('extracted_files')

In [ ]:
# Scraping and saving content for each URL
for index, row in df.iterrows():
    content_files(row['URL_ID'], row['URL'])

In [5]:
stopwords_d = "StopWords"
ext_dir = "extracted_files"

stop_words = set()
for files in os.listdir(stopwords_d):
  with open(os.path.join(stopwords_d,files),'r',encoding='ISO-8859-1') as f:
    stop_words.update(set(f.read().splitlines()))

In [ ]:
stop_words

In [6]:

docs = []
for file in os.listdir(ext_dir):
  with open(os.path.join(ext_dir, file), 'r', encoding='utf-8') as f:
    text = f.read()
    text = re.sub(r'[^\w\s.]','',text)
#tokenize the given text file
    words = word_tokenize(text)
# remove the stop words from the tokens
    filtered_text = [word for word in words if word.lower() not in stop_words]
# add each filtered tokens of each file into a list
    docs.append(filtered_text)

    sentences = text.split('.')
    num_sentences = len(sentences)
    words = [word  for word in text.split() if word.lower() not in stop_words ]
    num_words = len(words)

In [ ]:
docs

In [7]:
pos_words = set()
neg_words = set()

In [8]:
master_dir = "MasterDictionary"

for file in os.listdir(master_dir):
  if file =='positive-words.txt':
    with open(os.path.join(master_dir,file),'r',encoding='ISO-8859-1') as f:
      pos_words.update(f.read().splitlines())
  else:
    with open(os.path.join(master_dir,file),'r',encoding='ISO-8859-1') as f:
      neg_words.update(f.read().splitlines())

In [ ]:
pos_words

In [ ]:
neg_words

In [9]:
positive_words = []
negative_words =[]
pos_score = []
neg_score = []
polarity_score = []
subj_score = []
avg_sentence_length = []
perc_of_complex_words  =  []
fog_index = []
complex_word_count =  []
avg_syll_word_count =[]
word_count = []
average_word_length = []
pp_count = []

In [10]:
for i in range(len(docs)):
  positive_words.append([word for word in docs[i] if word.lower() in pos_words])
  negative_words.append([word for word in docs[i] if word.lower() in neg_words])
  pos_score.append(len(positive_words[i]))
  neg_score.append(len(negative_words[i]))
  polarity_score.append((pos_score[i] - neg_score[i]) / ((pos_score[i] + neg_score[i]) + 0.000001))
  subj_score.append((pos_score[i] + neg_score[i]) / ((len(docs[i])) + 0.000001))

In [11]:
def file_metrics(file):
    for file in os.listdir(ext_dir):
      with open(os.path.join(ext_dir, file), 'r', encoding='utf-8') as f:
        text = f.read()
        sentences = text.split('.')
        num_sentences = len(sentences)
        words = [word  for word in text.split() if word.lower() not in stop_words]
        num_words = len(words)

        #complex words
        complex_words = []
        for word in words:
          vowels = 'aeiou'
          syllable_count_word = sum(1 for letter in word if letter.lower() in vowels)
          if syllable_count_word > 2:
            complex_words.append(word)


        #syllable words
        syllable_count = 0
        syllable_words =[]
        for word in words:
          if word.endswith('es'):
            word = word[:-2]
          elif word.endswith('ed'):
            word = word[:-2]
          vowels = 'aeiou'
          syllable_count_word = sum( 1 for letter in word if letter.lower() in vowels)
          if syllable_count_word >= 1:
            syllable_words.append(word)
            syllable_count += syllable_count_word


        avg_sentence_len = num_words / num_sentences
        avg_syll_word_count = syllable_count / len(syllable_words)
        perc_complex_words  =  len(complex_words) / num_words
        fog_index = 0.4 * (avg_sentence_len + perc_complex_words)

        return avg_sentence_len, len(complex_words), avg_syll_word_count, perc_complex_words, fog_index

In [12]:
def len_words(file):
  with open(os.path.join(ext_dir, file), 'r', encoding='utf-8') as f:
    text = f.read()
    text = re.sub(r'[^\w\s]', '' , text)
    words = [word  for word in text.split() if word.lower() not in stop_words]
    length = sum(len(word) for word in words)
    average_word_length = length / len(words)
  return len(words),average_word_length

In [13]:
def personal_pronouns(file):
  with open(os.path.join(ext_dir,file), 'r', encoding='utf-8') as f:
    text = f.read()
    personal_pronouns = ["I", "we", "my", "ours", "us"]
    count = 0
    for pronoun in personal_pronouns:
      count += len(re.findall(r"\b" + pronoun + r"\b", text)) # \b is used to match word boundaries
  return count

In [14]:
for i in os.listdir(ext_dir):
    a,b,c,d,e = file_metrics(i)
    avg_sentence_length.append(a)
    complex_word_count.append(b)
    avg_syll_word_count.append(c)
    perc_of_complex_words.append(d)
    fog_index.append(e)

    a, b = len_words(i)
    word_count.append(a)
    average_word_length.append(b)

    x = personal_pronouns(i)
    pp_count.append(x)

In [19]:
variables = {'POSITIVE SCORE':pos_score,
            'NEGATIVE SCORE':neg_score,
            'POLARITY SCORE':polarity_score,
            'SUBJECTIVITY SCORE':subj_score,
            'AVG SENTENCE LENGTH':avg_sentence_length,
            'PERCENTAGE OF COMPLEX WORDS':perc_of_complex_words,
            'FOG INDEX':fog_index,
            'AVG NUMBER OF WORDS PER SENTENCE':avg_sentence_length,
            'COMPLEX WORD COUNT':complex_word_count,
            'WORD COUNT':word_count,
            'SYLLABLE PER WORD':avg_syll_word_count,
            'PERSONAL PRONOUNS':pp_count,
            'PERSONAL PRONOUNS':average_word_length}

existing_df = pd.read_excel('Output Data Structure.xlsx')
first_two_columns = existing_df.iloc[:, :2]
new_df = pd.DataFrame(variables)
final_df = pd.concat([first_two_columns, new_df], axis=1)
final_df.to_csv('output.csv', index=False)
